<h1 style = "text-align: center">Collaborative Filtering Data Model</h1>

<h3 style = "text-align: center">Food.com Recipe Recommender - SOEN 471 (Big Data Analytics)</h3>

## Objective:
The objective of this notebook is to create a recommender system data model that recommend recipes based on user preferences using collaborative filtering.

In [1]:
import os
import json
import numpy as np
import pandas as pd 
import dask.array as da
import dask.dataframe as dd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

# accessing directory
for dirname, _, filenames in os.walk('./clean_data'):
    for filename in filenames:
        os.path.join(dirname, filename)

## Reading files:

In [2]:
# Removing column titled "Unnamed: 0"
training = dd.read_csv("./clean_data/interactions_TRAIN.csv").drop("Unnamed: 0", axis=1)
#testing = dd.read_csv("./clean_data/interactions_TEST.csv").drop("Unnamed: 0", axis=1)
recipes = dd.read_csv("./clean_data/recipes.csv").drop("Unnamed: 0", axis=1)

## Print Training Data

In [3]:
training.head()

,user_id,recipe_id,date,rating,review
0,59801,8060,2007-12-11,5,This really is a great basic meatball recipe. ...
1,104295,38353,2005-02-08,5,Very yummy. I quartered the recipe and theref...
2,58104,162999,2007-03-05,5,OK this may be a great Canadian drink! But whe...
3,2000207498,2886,2015-05-16,5,Great recipe. I made a lot of changes though a...
4,41695,27082,2002-06-10,4,"Very easy to make and I liked it, my kids didn..."


In [4]:
recipes[recipes['recipe_id'] == 0].compute()

,name,recipe_id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,n_ingredients,Calories,Total_fat_PDV,Sugar_PDV,Sodium_PDV,Protein_PDV,Saturated_fat_PDV,Carbohydrates_PDV


## Print Recipes Data

In [5]:
recipes.head()

,name,recipe_id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,n_ingredients,Calories,Total_fat_PDV,Sugar_PDV,Sodium_PDV,Protein_PDV,Saturated_fat_PDV,Carbohydrates_PDV
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


## Add user rating column per recipe

In [6]:
recipes_ratings = dd.merge(recipes, training)
recipes_ratings.head(15)

,name,recipe_id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,...,Total_fat_PDV,Sugar_PDV,Sodium_PDV,Protein_PDV,Saturated_fat_PDV,Carbohydrates_PDV,user_id,date,rating,review
0,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",...,17.0,10.0,2.0,14.0,8.0,20.0,296809,2008-06-11,4,"This is an excellent, easy-fix side-dish w/ton..."
1,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",...,1.0,337.0,23.0,3.0,0.0,28.0,1310146,2011-08-21,5,"I wasn't sure how this was going to turn out, ..."
2,apple a day milk shake,5289,0,1533,1999-12-06,"['15-minutes-or-less', 'time-to-make', 'course...",4,"['combine ingredients in blender', 'cover and ...",NaN,"['milk', 'vanilla ice cream', 'frozen apple ju...",...,10.0,55.0,3.0,9.0,20.0,7.0,192972,2007-05-10,5,"We found this to be very good, and we make it ..."
3,bananas 4 ice cream pie,70971,180,102353,2003-09-10,"['weeknight', 'time-to-make', 'course', 'main-...",8,"['crumble cookies into a 9-inch pie plate , or...",NaN,"['chocolate sandwich style cookies', 'chocolat...",...,254.0,1306.0,111.0,127.0,431.0,220.0,20754,2003-11-24,5,This pie was fantastic!! Tasted just like a b...
4,bananas 4 ice cream pie,70971,180,102353,2003-09-10,"['weeknight', 'time-to-make', 'course', 'main-...",8,"['crumble cookies into a 9-inch pie plate , or...",NaN,"['chocolate sandwich style cookies', 'chocolat...",...,254.0,1306.0,111.0,127.0,431.0,220.0,35106,2005-10-31,5,Big hit with the family! I crushed the oreo co...
5,beat this banana bread,75452,70,15892,2003-11-04,"['weeknight', 'time-to-make', 'course', 'main-...",12,"['preheat oven to 350 degrees', 'butter two 9x...",from ann hodgman's,"['sugar', 'unsalted butter', 'bananas', 'eggs'...",...,160.0,976.0,107.0,62.0,310.0,138.0,217716,2005-07-27,4,"a very tasty bread, I would only reduce the su..."
6,better than sex strawberries,42198,1460,41531,2002-10-03,"['weeknight', 'time-to-make', 'course', 'main-...",8,['crush vanilla wafers into fine crumbs and li...,simple but sexy. this was in my local newspape...,"['vanilla wafers', 'butter', 'powdered sugar',...",...,66.0,199.0,10.0,10.0,117.0,28.0,88342,2003-08-04,0,You shouldn't be recommending eating raw eggs ...
7,boat house collard greens,107517,525,137696,2005-01-03,"['time-to-make', 'main-ingredient', 'preparati...",7,"['put prepared greens in large pot', 'add wate...",my boss gave me this recipe several years ago....,"['collard greens', 'brown sugar', 'molasses', ...",...,0.0,202.0,9.0,6.0,0.0,21.0,588703,2007-12-29,5,Really great. I made these for my English husb...
8,calm your nerves tonic,39959,5,37449,2002-09-10,"['15-minutes-or-less', 'time-to-make', 'prepar...",6,"['combine herbs', 'dosage: one-half teaspoonfu...",this will prove a blessing to everyone who tak...,"['gentian root', 'scullcap herb', 'burnet root...",...,0.0,0.0,0.0,1.0,0.0,0.0,65056,2005-05-21,5,This certainly is a blessing! I had to omit t...
9,chicken lickin good pork chops,63986,500,14664,2003-06-06,"['weeknight', 'time-to-make', 'course', 'main-...",5,"['dredge pork chops in mixture of flour , salt...",here's and old standby i enjoy from time to ti...,"['lean pork chops', 'flour', 'salt', 'dry must...",...,8.0,0.0,26.0,5.0,4.0,3.0,592866,2007-09-20,5,Excellent! A friend gave me this recipe years ...


## Making User-Item Matrix

In [ ]:
training = training.head(90000)
ratings_matrix = training.pivot_table(index='recipe_id',columns='user_id',values='rating')
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(15)

In [ ]:
ratings_matrix.loc[27082][41695]

## Performing Cosine Similarity

In [ ]:
recipe_similarity = cosine_similarity(ratings_matrix)
np.fill_diagonal(recipe_similarity, 0) 
recipe_similarity

## Making Ratings Matrix

In [ ]:
ratings_matrix = dd.from_array(recipe_similarity)
ratings_matrix.head(15)

## Recommending based on Recipe ID

In [ ]:
try:
    user_inp=137739
    inp=recipes[recipes['title']==user_inp].index.tolist()
    inp=inp[0]
    
    recipes['similarity'] = ratings_matrix.iloc[inp]
    recipes.head(5)
    
except:
    print("Sorry, the recipe is not in the database!")
    
print("Recommended recipes based on your choice of ",user_inp ,": \n", recipes.sort_values( ["similarity"], ascending = False )[1:10])